# Goal of the project

The goal of this project is to control a 2D quadrotor to reach a target while avoiding obstacles using a learned policy. You wil have to create your own Custom environment using the [stable_baselines3](https://stable-baselines3.readthedocs.io/en/master/) library and train a RL agent using PPO.

## 2D quadrotor

The quadrotor is depicted in the following figure
<img src='quadrotor.png' width="300">


The quadrotor model is written as
$$\begin{align} 
\dot{p_x} &= v_x\\
m \dot{v}_x &= - (u_1 + u_2) \sin \theta \\ 
\dot{p_y} &= v_y\\
m \dot{v}_y &= (u_1 + u_2) \cos \theta  - m g\\
\dot{\theta} &= \omega\\
I \dot{\omega} &= r (u_1 - u_2) \end{align}$$
where $p_x$ is the horizontal and $p_y$ the vertical positions of the quadrotor and $\theta$ is its orientation with respect to the horizontal plane. $v_x$ and $v_y$ are the linear velocities and $\omega$ is the angular velocity of the robot. $u_1$ and $u_2$ are the forces produced by the rotors (our control inputs). $m$ is the quadrotor mass, $I$ its moment of inertia (a scalar), $r$ is the distance from the center of the robot frame to the propellers and $g$ is the gravity constant. To denote the entire state, we will write $x = [p_x, v_x, p_y, v_y, \theta, \omega]^T$ - we will also write $u = [u_1, u_2]^T$.

The module ```quadrotor.py``` defines the problem and provides all the useful information about the robot and methods to simulate and animate it as shown below.

You can access the different parameters of the model in the following way:

In [60]:
import quadrotor

print("Mass    =", quadrotor.MASS)
print("Length  =", quadrotor.LENGTH)
print("Inertia =", quadrotor.INERTIA)
print("Dt      =", quadrotor.DT)
print("state size   =", quadrotor.DIM_STATE)
print("control size =", quadrotor.DIM_CONTROL)

Mass    = 0.5
Length  = 0.15
Inertia = 0.1
Dt      = 0.04
state size   = 6
control size = 2


The goal of this project is to learn a policy that can move the robot from any point to the red dot ($x^{\star} = [2, 0, 0, 0, 0, 0]^T$) while avoiding thee obstacles. The obstacles are represented by the black circles in the animation. You can check if the drone is in collision with an obstacle using the function ```quadrotor.check_collision```. 

## Create a RL environment
Using the [stable_baselines3](https://stable-baselines3.readthedocs.io/en/master/), create a [custom RL environment](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html) environment. You will have to follow the following steps:

1. Implement a step function than contrains the dynamics (you are free to use the ```quadrotor.next_state```) and a reward function. To speed-up the training, make sure to add a gravity compensation term in your dynamics (i.e. the drone should stay in place when the policy outputs zeros).
   The reward should be made of three terms:
   
- A positive term to incentivize the quadrotor to reach the target. You can start with a reward bounded between 0 and 1, e.g.
   $\operatorname{exp}(-\frac{1}{2} (x - x^{\star})Q(x - x^{\star}) -\frac{1}{2} (u - u_{\text{gravity}})R(u - u_{\text{gravity}}))$

   
- A large negative penality(e.g. -100)  if the robot get out of the following bounds:
$ p_x \in [-4, 4], \quad v_x \in [-10, 10] , \quad p_y \in [-4, 4] , \quad v_y \in [-10, 10] , \quad \theta \in [-2 \pi, 2 \pi] , \quad \omega \in [-10, 10] $.

 - A negative penalty if the robot hits the obstacle, e.g. -1. You should use the ```quadrotor.check_collision```.

Keep in mind that, in RL, the goal is to maximize a reward (and not minimize a cost like in Optimal Control).

2. Implement a reset function that initializes the state randomly. You can sample uniformly between $[-2, 2]$ for $p_x$ and $p_y$ and initialize the other terms to zero. Make sure to reject samples that are colliding with the obstacles using the ```quadrotor.check_collision```.

3. In the step function, stop the environment using ```truncated``` after 200 steps (Here is an [example](https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/5_custom_gym_env.ipynb)).

4. In the step function, stop the environment if the drone goes outsite of the provided bounds using ```terminated``` (Here is an [example](https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/5_custom_gym_env.ipynb)).

5. Make sure that your environment is well defined using the ```check_env``` function.
   
## Training a policy with PPO   
Train a policy with PPO and use the learned policy to define a controller. Make sure that you can reach the target while avoiding the obstacles starting from $x_0 = [-2, 0, 0, 0 ,0, 0]$



Please submit your code (as runnable Jupyter Notebook), a pdf report and an mp4 video. In the report, explain your reward design and provide plots showing the trajectory of the quadrotor. The mp4 video should show the quadrotor animation starting from $x_0 = [-2, 0, 0, 0 ,0, 0]$. You can save your animation in the following way:


In [61]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython


def controller(x, t):
    return np.zeros(2)

x_init = np.array([-2, 0, 0., 0 ,0, 0])
horizon_length = 200
# t, state, u = quadrotor.simulate(x_init, controller, horizon_length)
# quadrotor.animate_robot(state, u, save_mp4=True)

In [62]:
# # N = 100
# # u = np.array([[0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT,
# #              0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT]] * N).T

# def controller(x, t):
#     return np.array([0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT,
#                      0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT])
#     # return np.zeros(2)


# x_init = np.array([-2, 0, 0., 0, 0, 0])
# horizon_length = 100
# t, state, u = quadrotor.simulate(x_init, controller, horizon_length)
# quadrotor.animate_robot(state, u, save_mp4=False)

In [63]:
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO
from time import strftime

In [64]:
class QuadrotorEnv(gym.Env):

    def __init__(self, goal=np.array([2, 0, 0, 0, 0, 0])):
        super().__init__()
        self.u_gravity = np.array([0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT,
                                   0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT])
        self.action_space = spaces.Box(
            low=-1e6, high=1e6,
            shape=(quadrotor.DIM_CONTROL, ), dtype=np.float32)
        # self.observation_space = spaces.Box(
        #     low=np.negative(np.array([4, 10, 4, 10, 2*np.pi, 10])),
        #     high=np.array([4, 10, 4, 10, 2*np.pi, 10]),
        #     shape=(quadrotor.DIM_STATE, ),
        #     dtype=np.float32)
        self.observation_space = spaces.Box(
            low=-1e6,
            high=1e6,
            shape=(quadrotor.DIM_STATE, ),
            dtype=np.float32)
        self.Q = np.diag(np.array([31.25, 0.03125, 31.25, 0.03125,
                                   3.125, 0.03125], dtype=np.float32))
        self.R = np.diag(np.array([0.75, 0.75], dtype=np.float32))
        self.goal = goal
        self.current_state = np.zeros(6)
        self.max_steps = 200
        self.current_step = 0  # track steps per episode
        self.reset()

    def step(self, action):
        # if action not in self.action_space:
        #     raise ValueError(
        #         f'The chosen action {action} does not exist in the action space {self.action_space}')
        # if observation not in self.observation_space:
        #     raise ValueError(
        #         f'The current observation {observation} does not exist in the observation space {self.observation_space}')

        action = action + self.u_gravity
        self.current_step += 1

        # Compute next state
        next_state = quadrotor.next_state(self.current_state, action)
        next_state = np.asarray(next_state, dtype=np.float32)

        # Check bounds and set terminated if outside
        if not self._within_bounds(next_state):
            terminated = True
            truncated = False
            reward = -100  # large negative penalty for going out of bounds
            return next_state, reward, terminated, truncated, {}

        # Check collision
        collided = quadrotor.check_collision(next_state)
        if collided:
            # Negative penalty for collision
            terminated = True
            truncated = False
            reward = -1
            return next_state, reward, terminated, truncated, {}

        # Check if goal reached
        # Instead of checking exact equality, check proximity:
        if np.linalg.norm(next_state - self.goal) < 0.1:  # threshold
            terminated = True
            truncated = False
            # You can give a high positive reward or rely on the reward function
            # defined in _get_reward
        else:
            terminated = False

        # Check step count for truncation
        if self.current_step >= self.max_steps:
            truncated = True
        else:
            truncated = False

        # Compute reward
        reward = self._get_reward(action, next_state)

        self.current_state = next_state
        return next_state, reward, terminated, truncated, {}

    def reset(self, seed=42, options=None):
        super().reset(seed=seed, options=options)
        p_x = self.np_random.uniform(-2, 2)
        p_y = self.np_random.uniform(-2, 2)
        v_x = 0.0
        v_y = 0.0
        theta = 0.0
        omega = 0.0
        init_state = np.array(
            [p_x, v_x, p_y, v_y, theta, omega], dtype=np.float32)

        # Check collision
        while quadrotor.check_collision(init_state):
            p_x = self.np_random.uniform(-2, 2)
            p_y = self.np_random.uniform(-2, 2)
            init_state = np.array([p_x, 0, p_y, 0, 0, 0], dtype=np.float32)

        info = {}
        init_state = np.asarray(init_state, dtype=np.float32)
        self.current_state = init_state
        self.current_step = 0
        return init_state, info

    def render(self):
        ...

    def close(self):
        ...

    def _get_reward(self, action, observation) -> float:
        if not self._within_bounds(observation):
            return -100
        elif quadrotor.check_collision(observation):
            return -1
        else:
            return np.exp(
                -((1/2) * ((observation - self.goal).T @
                           self.Q @ (observation - self.goal)))
                - ((1/2) * ((action - self.u_gravity).T @
                            self.R @ (action - self.u_gravity)))
            )

    def _within_bounds(self, state):
        p_x, v_x, p_y, v_y, theta, omega = state
        if p_x < -4 or p_x > 4:
            return False
        if v_x < -10 or v_x > 10:
            return False
        if p_y < -4 or p_y > 4:
            return False
        if v_y < -10 or v_y > 10:
            return False
        if theta < -2*np.pi or theta > 2*np.pi:
            return False
        if omega < -10 or omega > 10:
            return False
        return True


check_env(QuadrotorEnv(), warn=True)

In [65]:
# # N = 100
# env = QuadrotorEnv()
# quadrotor_model = PPO(policy='MlpPolicy', env=env, verbose=1,
#                       n_steps=horizon_length)
# quadrotor_model.learn(total_timesteps=1e4)
# quadrotor_model.save(
#     f'saved_models/quadrotor_model_{strftime("%Y%m%d_%H%M%S")}')

env = QuadrotorEnv()
quadrotor_model = PPO(
    policy='MlpPolicy',
    env=env,
    verbose=1,
    n_steps=horizon_length
)

quadrotor_model.learn(total_timesteps=1e5)  # consider a larger number of steps
quadrotor_model.save(
    f'saved_models/quadrotor_model_{strftime("%Y%m%d_%H%M%S")}')

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/shantanu/Documents/Academics/NYUMSMRFALL24/Reinforcement Learning and Optimal Control for Robotics (ROB-GY 6323)/Project 2/.venv/lib/python3.10/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 200`, after every 3 untruncated mini-batches, there will be a truncated mini-batch of size 8
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=200 and n_envs=1)
  warnings.warn(
/home/shantanu/Documents/Academics/NYUMSMRFALL24/Reinforcement Learning and Optimal Control for Robotics (ROB-GY 6323)/Project 2/.venv/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 43.2     |
|    ep_rew_mean     | -75.2    |
| time/              |          |
|    fps             | 753      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 200      |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 46.6         |
|    ep_rew_mean          | -61.7        |
| time/                   |              |
|    fps                  | 626          |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 400          |
| train/                  |              |
|    approx_kl            | 0.0035496324 |
|    clip_fraction        | 0.0125       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.84        |
|    explained_variance   | 0.00947      |
|    learning_r

In [66]:

def controller(x, t):
    return quadrotor_model.predict(observation=x)[0]


x_init = np.array([-2, 0, 0., 0, 0, 0])
horizon_length = 200
t, state, u = quadrotor.simulate(x_init, controller, horizon_length)
quadrotor.animate_robot(state, u, save_mp4=False)

Your browser does not support the video tag.

In [ ]:
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import quadrotor


class QuadrotorEnv(gym.Env):
    def __init__(self, goal=np.array([2, 0, 0, 0, 0, 0])):
        super().__init__()
        self.u_gravity = np.array([0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT,
                                   0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT])
        self.action_space = spaces.Box(
            low=-1e6, high=1e6,
            shape=(quadrotor.DIM_CONTROL, ), dtype=np.float32)

        # The observation space is large, you might consider bounding it more tightly
        self.observation_space = spaces.Box(
            low=-1e6,
            high=1e6,
            shape=(quadrotor.DIM_STATE, ),
            dtype=np.float32)

        self.Q = np.diag(np.array([31.25, 0.03125, 31.25, 0.03125,
                                   3.125, 0.03125], dtype=np.float32))
        self.R = np.diag(np.array([0.75, 0.75], dtype=np.float32))
        self.goal = goal
        self.max_steps = 200
        self.current_step = 0
        self.current_state = None

        # Initialize environment
        self.reset()

    def step(self, action):
        self.current_step += 1

        # Add gravity compensation
        compensated_action = action + self.u_gravity

        # Compute next state
        next_state = quadrotor.next_state(
            self.current_state, compensated_action)
        next_state = np.asarray(next_state, dtype=np.float32)

        # Check if out of bounds
        if not self._within_bounds(next_state):
            reward = -100
            terminated = True
            truncated = False
            return next_state, reward, terminated, truncated, {}

        # Check collision
        if quadrotor.check_collision(next_state):
            reward = -1
            terminated = True
            truncated = False
            return next_state, reward, terminated, truncated, {}

        # Check goal reached
        if np.linalg.norm(next_state - self.goal) < 0.1:
            # Give a nice bonus for reaching the goal
            reward = 10.0
            terminated = True
            truncated = False
            self.current_state = next_state
            return next_state, reward, terminated, truncated, {}

        # Check time steps for truncation
        truncated = (self.current_step >= self.max_steps)

        # If we got here, we're neither terminated nor truncated, so compute normal reward
        reward = self._get_reward(compensated_action, next_state)

        self.current_state = next_state
        return next_state, reward, False, truncated, {}

    def reset(self, seed=42, options=None):
        super().reset(seed=seed, options=options)
        self.current_step = 0

        # Sample a valid initial state
        p_x = self.np_random.uniform(-2, 2)
        p_y = self.np_random.uniform(-2, 2)
        v_x = 0.0
        v_y = 0.0
        theta = 0.0
        omega = 0.0
        init_state = np.array(
            [p_x, v_x, p_y, v_y, theta, omega], dtype=np.float32)

        # Ensure no collision at start
        while quadrotor.check_collision(init_state):
            p_x = self.np_random.uniform(-2, 2)
            p_y = self.np_random.uniform(-2, 2)
            init_state = np.array([p_x, 0, p_y, 0, 0, 0], dtype=np.float32)

        self.current_state = init_state
        info = {}
        return init_state, info

    def render(self):
        pass

    def close(self):
        pass

    def _get_reward(self, action, observation):
        # Here we assume we're not out of bounds or colliding (already checked).
        # Compute continuous reward
        return np.exp(
            -0.5 * ((observation - self.goal).T @
                    self.Q @ (observation - self.goal))
            - 0.5 * ((action - self.u_gravity).T @
                     self.R @ (action - self.u_gravity))
        )

    def _within_bounds(self, state):
        p_x, v_x, p_y, v_y, theta, omega = state
        if p_x < -4 or p_x > 4:
            return False
        if v_x < -10 or v_x > 10:
            return False
        if p_y < -4 or p_y > 4:
            return False
        if v_y < -10 or v_y > 10:
            return False
        if theta < -2*np.pi or theta > 2*np.pi:
            return False
        if omega < -10 or omega > 10:
            return False
        return True


# Check if the environment is consistent
check_env(QuadrotorEnv(), warn=True)

env = QuadrotorEnv()
quadrotor_model = PPO(
    policy='MlpPolicy',
    env=env,
    verbose=1,
    n_steps=horizon_length
)

quadrotor_model.learn(total_timesteps=1e5)  # consider a larger number of steps
quadrotor_model.save(
    f'saved_models/quadrotor_model_{strftime("%Y%m%d_%H%M%S")}')


def controller(x, t):
    return quadrotor_model.predict(observation=x)[0]


x_init = np.array([-2, 0, 0., 0, 0, 0])
horizon_length = 200
t, state, u = quadrotor.simulate(x_init, controller, horizon_length)
quadrotor.animate_robot(state, u, save_mp4=False)